In [10]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pickle
from tensorflow.keras.metrics import AUC, Precision, Recall
from llm_api import get_sentiment_and_emotion
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score, precision_score, recall_score, roc_auc_score


# Load test data
df = pd.read_csv("goemotions_test_data.csv")



# Get the one-hot encoded labels from the original DataFrame
original_columns = df.columns.tolist()
num_label_columns = 27
label_column_names = original_columns[-num_label_columns:]

# Create test DataFrame with texts and labels
goemotions_valid_df = pd.DataFrame({
    'texts': df['texts'].tolist()
})

# Add label columns
for i, col in enumerate(label_column_names):
    goemotions_valid_df[col] = df[col].values

# Shuffle the DataFrame and take only 100 rows for testing
goemotions_valid_df = goemotions_valid_df.sample(frac=1, random_state=42).reset_index(drop=True).head(100)

# Reorder columns 
goemotions_valid_df = goemotions_valid_df[['texts','processed_text', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral']]


In [11]:
goemotions_valid_df.head()

,texts,processed_text,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,"If that's true, then you can't call what I wou...","if that is true, then you cannot call what i w...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Lol. Yes, the contractors know","lol. yes, the contractors know",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Haha I was thinking Utah but... Florida makes ...,haha i was thinking utah but.. florida makes s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,"Yeah, because it’s soooo arduous to look at so...","yeah, because it is soo arduous to look at som...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,That was intentional as fuck. I don’t know how...,that was intentional as fuck. i do not know ho...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Function to find the first non-zero emotion label for each row
def get_true_emotion(row):
    # Start from index 2 (third column) which is 'anger'
    for col in row.index[2:]:
        if row[col] == 1:
            return col.lower()
    return None

# Apply function to get true emotion labels
goemotions_valid_df['true_emotion'] = goemotions_valid_df.apply(get_true_emotion, axis=1)


In [16]:


# Map LLM function over texts to get sentiment and emotion
sentiment_emotion = goemotions_valid_df['texts'].apply(get_sentiment_and_emotion)
goemotions_valid_df['sentiment'] = sentiment_emotion.apply(lambda x: x[0])
goemotions_valid_df['predicted_emotion'] = sentiment_emotion.apply(lambda x: x[1])

# Clean sentiment and emotion columns to lowercase
goemotions_valid_df['sentiment'] = goemotions_valid_df['sentiment'].str.lower()
goemotions_valid_df['predicted_emotion'] = goemotions_valid_df['predicted_emotion'].str.lower()

# Show the first few rows with new columns
print("\nFirst few rows of processed data:")
print(goemotions_valid_df.head())


First few rows of processed data:
                                               texts  \
0  If that's true, then you can't call what I wou...   
1                     Lol. Yes, the contractors know   
2  Haha I was thinking Utah but... Florida makes ...   
3  Yeah, because it’s soooo arduous to look at so...   
4  That was intentional as fuck. I don’t know how...   

                                      processed_text  anger  annoyance  \
0  if that is true, then you cannot call what i w...    0.0        0.0   
1                     lol. yes, the contractors know    0.0        0.0   
2  haha i was thinking utah but.. florida makes s...    0.0        0.0   
3  yeah, because it is soo arduous to look at som...    0.0        0.0   
4  that was intentional as fuck. i do not know ho...    0.0        0.0   

   approval  caring  confusion  curiosity  desire  disappointment  ...  pride  \
0       1.0     0.0        0.0        0.0     0.0             0.0  ...    0.0   
1       0.0     0.0  

In [17]:
goemotions_valid_df

,texts,processed_text,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,...,pride,realization,relief,remorse,sadness,surprise,neutral,true_emotion,sentiment,predicted_emotion
0,"If that's true, then you can't call what I wou...","if that is true, then you cannot call what i w...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,approval,negative,anger
1,"Lol. Yes, the contractors know","lol. yes, the contractors know",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,curiosity,negative,amusement
2,Haha I was thinking Utah but... Florida makes ...,haha i was thinking utah but.. florida makes s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,neutral,negative,confusion
3,"Yeah, because it’s soooo arduous to look at so...","yeah, because it is soo arduous to look at som...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,gratitude,negative,frustration
4,That was intentional as fuck. I don’t know how...,that was intentional as fuck. i do not know ho...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,realization,negative,frustration
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,I can’t help but think it’s BS because he foun...,i cannot help but think it is bs because he fo...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,caring,negative,frustration
96,These are some of the lamest threatening lette...,these are some of the lamest threatening lette...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,negative,anger
97,Such a beautiful smile! I can tell from your e...,such a beautiful smile! i can tell from your e...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,neutral,positive,joy
98,RNG gear is a solid no from me.,rng gear is a solid no from me.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,optimism,negative,dislike


In [18]:
goemotions_valid_df.to_csv('goemotions_valid_df.csv', index=False)

In [19]:
# Align predicted and true emotions
y_true = goemotions_valid_df['true_emotion']
y_pred = goemotions_valid_df['predicted_emotion']

In [20]:
# Filter to only those predictions that are in the label set
labels = ['anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral']
mask = y_pred.isin(labels)
y_true = y_true[mask]
y_pred = y_pred[mask]

In [21]:
y_true

0        approval
2         neutral
6            fear
8         neutral
9     disapproval
         ...     
91            NaN
92            NaN
94            joy
96            NaN
97        neutral
Name: true_emotion, Length: 72, dtype: object

In [22]:
# Create confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
fig, ax = plt.subplots(figsize=(12, 12))
disp.plot(ax=ax, xticks_rotation=90)
plt.title('Confusion Matrix for Emotion Classification')
plt.savefig('goemotions_llm_confusion_matrix.png')
plt.tight_layout()
plt.show()

# Save the processed DataFrame to CSV
goemotions_valid_df.to_csv('goemotions_llm_results.csv', index=False)
print("\nResults saved to 'goemotions_llm_results.csv'")


TypeError: '<' not supported between instances of 'str' and 'float'